In [6]:
import pandas as pd
import os
from dotenv import load_dotenv
from shutil import copy2
import psycopg2

In [3]:
def date_changer(df):

    df['Date'] = pd.to_datetime(df['Date'])

    return df['Date']

In [10]:
base_folder = '../data'

for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)

    if 'Chart data.csv' in os.listdir(folder_path):
        df = pd.read_csv(f'{folder_path}/Chart data.csv')

        df['Date'] = date_changer(df)

        df.to_csv(f'../database_file/{folder}.csv', index=False)

    elif len(os.listdir(folder_path)) == 1:
        source_file = os.path.join(folder_path, os.listdir(folder_path)[0])
        destination_path = '../database_file'

        source_name =  os.path.join('../database_file', os.listdir(folder_path)[0])
        destination_name = os.path.join('../database_file', f'{folder}.csv')

        copy2(source_file, destination_path)
        os.rename(source_name, destination_name)

    else:
        source_file = os.path.join(folder_path, 'Totals.csv')
        destination_path = '../database_file'

        source_name =  os.path.join('../database_file', 'Totals.csv')
        destination_name = os.path.join('../database_file', f'{folder}.csv')

        copy2(source_file, destination_path)
        os.rename(source_name, destination_name)


In [11]:
load_dotenv('../.env')
host=os.getenv('HOST')
port=os.getenv('PORT')
username=os.getenv('USERNAME')
password=os.getenv('PASSWORD')
database=os.getenv('DATABASE')


def import_csv_to_table(csv_file, table_name):

    try:
        conn = psycopg2.connect(host=host, port=port, user=username, password=password, database=database)
        cur = conn.cursor()


        with open(csv_file, 'r') as f:

            next(f)
            cur.copy_from(f, table_name, sep=';', null='')

        conn.commit()
        cur.close()

    except (Exception, psycopg2.Error) as error:
        print(f'Error importing data fromm{csv_file}: {error}')

SyntaxError: unexpected EOF while parsing (3862444657.py, line 10)

In [ ]:
csv_folder = '../database_file'
for filename in os.listdire(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)
        table_name = filename[:-4]
        import_csv_to_table(csv_file,table_name)